In [1]:
from gevent import monkey
monkey.patch_all(thread=False)
from gevent.queue import Queue
import gevent

# = = = = = = = = = = = = = = =

from IPython.display import clear_output
from datetime import datetime

In [2]:
import pandas as pd

output_correct, output_error = pd.DataFrame(), pd.DataFrame()

# = = = = = = = = = = = = = = =

input_ = pd.read_excel('./3.url_2.xlsx',
                       header=0,
                       dtype=str).fillna('')

input_['Part No'] = input_['Part No'].astype(int)

print(f'总数量：{len(input_)}')
print()

# = = = = = = = = = = = = = = =

work = Queue()
for a in range(len(input_)):
    work.put_nowait(a)

# = = = = = = = = = = = = = = =

def crawler():
    global output_correct, output_error

    while not work.empty():
        a = work.get_nowait()

        # = = = = = = = = = = = = = = =

        crawler_status = 'error'

        # = = = = = = = = = = = = = = =

        import requests

        import sys
        sys.path.append('../00.Tools')
        from crawler_configuration import get_header, get_proxy

        try:
            b = 0
            while True:

                while True:
                    b += 1
                    try:
                        resp = requests.get(input_.loc[a, 'Url'],
                                            headers=get_header(),
                                            proxies=get_proxy(),
                                            timeout=(10, 10))
                        break
                    except:
                        continue

                if resp.status_code == 200:
                    break

            # = = = = = = = = = = = = = = =

            from bs4 import BeautifulSoup
            from lxml import etree

            soup = BeautifulSoup(resp.text, 'lxml')
            html = etree.HTML(str(soup))

            # = = = = = = = = = = = = = = =

            title = html.xpath('//head/title/text()')[0].strip()

            # = = = = = = = = = = = = = = =

            subtitle = html.xpath('//h2[@itemprop="brand"]/a/span[@itemprop="name"]/text()')[0].strip()

            # = = = = = = = = = = = = = = =

            part_number = html.xpath('//dd[@itemprop="sku"]/text()')[0].strip()

            # = = = = = = = = = = = = = = =

            part_type = html.xpath('//dd[@data-product-parttype]/text()')[0].strip()

            # = = = = = = = = = = = = = = =

            dict_application_summary = {}
            list_application_summary = html.xpath('//meta[@name="description"]/@content')[0].strip().removeprefix(f'{subtitle},').removeprefix(f'{part_type},').split(';') if html.xpath('//meta[@name="description"]/@content') else ''
            for i, application_summary in zip(range(len(list_application_summary)), list_application_summary):
                dict_application_summary[str(i)] = application_summary.strip()

            # = = = = = = = = = = = = = = =

            dict_src = {}
            list_src = html.xpath('//li[contains(@class, "productView-thumbnail")]/a[@class="productView-thumbnail-link"]/@href')
            for i, src in zip(range(len(list_src)), list_src):
                dict_src[str(i)] = src.strip()

            # = = = = = = = = = = = = = = =

            import json

            dict_info = {}
            list_dict = json.loads(html.xpath('//div[@id="addon"]/@data-custom-fields')[0])
            for i, dict_ in zip(range(len(list_dict)), list_dict):
                dict_info[str(i)] = {dict_['name'].strip(): dict_['value'].strip()}

            # = = = = = = = = = = = = = = =

            bundle = resp.text.split('window.__webpack_public_path__ = "')[1].split('";</script>')[0]

            # = = = = = = = = = = = = = = =

            df_temp = pd.DataFrame([{'Part No': input_.loc[a, 'Part No'],
                                     'Title': title,
                                     'Subtitle': subtitle,
                                     'Part Number': part_number,
                                     'Part Type': part_type,
                                     'Json_Application_Summary': json.dumps(dict_application_summary),
                                     'Bundle': bundle,
                                     'Json_Info': json.dumps(dict_info),
                                     'Url': input_.loc[a, 'Url'],
                                     'Json_Src': json.dumps(dict_src)}])

            output_correct = pd.concat([output_correct, df_temp], ignore_index=True).fillna('')

            # = = = = = = = = = = = = = = =

            crawler_status = 'ok'
        except:
            df_temp = pd.DataFrame([{'Part No': input_.loc[a, 'Part No'],
                                     'Url': input_.loc[a, 'Url']}])

            output_error = pd.concat([output_error, df_temp], ignore_index=True).fillna('')

        # = = = = = = = = = = = = = = =

        if work.qsize() % 1_000 == 0 and work.qsize() != 0:
            clear_output()

        print(f'''[状态：{crawler_status}，尝试次数：{b}] - {input_.loc[a, 'Part No']}.{input_.loc[a, 'Url']}\n[剩余数量：{work.qsize()}] - [当前时间：{datetime.now().strftime('%H:%M:%S')}]\n''')

# = = = = = = = = = = = = = = =

list_task = []
for _ in range(12):
    task = gevent.spawn(crawler)
    list_task.append(task)
gevent.joinall(list_task)

print('输出ing...')
print()
if not output_correct.empty:
    output_correct = output_correct.sort_values(by=['Part No'],
                                                ascending=[True],
                                                ignore_index=True)
    output_correct.to_excel(f'''./4.part-{datetime.now().strftime('%Y%m%d_%H%M%S')}.xlsx''', index=False)
if not output_error.empty:
    output_error = output_error.sort_values(by=['Part No'],
                                            ascending=[True],
                                            ignore_index=True)
    output_error.to_excel('./part_error.xlsx', index=False)
    print('爬虫存在error')
    print()
print('Done ~')

总数量：107

[状态：ok，尝试次数：1] - 86.https://www.cardone.com/product/abs-control-module-12-10304
[剩余数量：95] - [当前时间：09:34:52]

[状态：ok，尝试次数：1] - 4851.https://www.cardone.com/product/brake-caliper-18-p5610
[剩余数量：94] - [当前时间：09:34:52]

[状态：ok，尝试次数：1] - 24.https://www.cardone.com/product/abs-control-module-12-10219
[剩余数量：93] - [当前时间：09:34:52]

[状态：ok，尝试次数：1] - 165.https://www.cardone.com/product/abs-control-module-12-17412
[剩余数量：92] - [当前时间：09:34:52]

[状态：ok，尝试次数：1] - 94.https://www.cardone.com/product/abs-control-module-12-10317
[剩余数量：91] - [当前时间：09:34:52]

[状态：ok，尝试次数：1] - 115.https://www.cardone.com/product/abs-control-module-12-10362
[剩余数量：90] - [当前时间：09:34:52]

[状态：ok，尝试次数：1] - 435.https://www.cardone.com/product/power-brake-booster-53-7641
[剩余数量：89] - [当前时间：09:34:53]

[状态：ok，尝试次数：1] - 4239.https://www.cardone.com/product/brake-caliper-18-b5649
[剩余数量：88] - [当前时间：09:34:53]

[状态：ok，尝试次数：1] - 70.https://www.cardone.com/product/abs-control-module-12-10280
[剩余数量：87] - [当前时间：09:34:53]

[状态：ok，尝试次数：1